# For Alaska study area, build yearly HLS fmask cubes and save to NetCDF

In [1]:
import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import STAC_Utils as sutil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65354,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:65392,Total threads: 10
Dashboard: http://127.0.0.1:65396/status,Memory: 0 B
Nanny: tcp://127.0.0.1:65357,


In [3]:
area = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\Fmask_V1\AA\Alaska Snow Depth\Extent\Alaska_snowSites.shp"
year = 2024 # 2018, 2019, 2020, 2021, 2022, 2023, 2024
start = str(year) + '-01-01'
end =  str(year) + '-12-31'
bands = ['Fmask']
chunksize = (1, -1, -1, -1)
sceneCloudThresh = 100
proj = 'poly'
verbose = True

In [4]:
%%time
fmask = sutil.buildHLS(area, start, end, bands, chunksize, sceneCloudThresh, proj, verbose)
fmask

CPU times: total: 1.67 s
Wall time: 25.2 s


<xarray.DataArray 'stackstac-c5926b00bdc2dd1977b0699da7910e9d' (time: 245,
                                                                band: 1,
                                                                y: 2436, x: 1581)> Size: 944MB
dask.array<getitem, shape=(245, 1, 2436, 1581), dtype=uint8, chunksize=(1, 1, 2436, 1581), chunktype=numpy.ndarray>
Coordinates:
  * band           (band) <U5 20B 'Fmask'
  * x              (x) float64 13kB 3.995e+05 3.996e+05 ... 4.469e+05 4.469e+05
  * y              (y) float64 19kB 7.679e+06 7.679e+06 ... 7.606e+06 7.606e+06
    constellation  (time) <U3 3kB 'L30' 'L30' 'S30' 'S30' ... 'S30' 'L30' 'L30'
  * time           (time) datetime64[ns] 2kB 2024-03-01 ... 2024-10-11
Attributes:
    spec:        RasterSpec(epsg=32606, bounds=(399510, 7606260, 446940, 7679...
    crs:         epsg:32606
    transform:   | 30.00, 0.00, 399510.00|\n| 0.00,-30.00, 7679340.00|\n| 0.0...
    resolution:  30

In [5]:
# Oops error related to: https://forum.earthdata.nasa.gov/viewtopic.php?t=6086&sid=94f3a3b5d6824851af8c81263cc15ec5
# Fixed by updating HLS collection: https://lpdaac.usgs.gov/news/important-update-to-cmr-stac-new-identifier-and-search-parameter-format/

In [6]:
%%time
fmask = pputil.loadXR(fmask) # Load here, like done in observationAvailabilityHLS() 
fmask

CPU times: total: 13.6 s
Wall time: 2min 22s


<xarray.DataArray 'stackstac-c5926b00bdc2dd1977b0699da7910e9d' (time: 245,
                                                                band: 1,
                                                                y: 2436, x: 1581)> Size: 944MB
array([[[[208, 208, 208, ..., 255, 255, 255],
         [208, 208, 208, ..., 255, 255, 255],
         [208, 208, 208, ..., 255, 255, 255],
         ...,
         [208, 208, 208, ..., 208, 208, 208],
         [208, 208, 208, ..., 208, 208, 208],
         [208, 208, 208, ..., 208, 208, 212]]],


       [[[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 208, 208, 208],
         [255, 255, 255, ..., 208, 208, 208],
         [255, 255, 255, ..., 208, 208, 208]]],


       [[[130, 130, 130, ..., 255, 255, 255],
         [130, 130, 130, ..., 255, 255, 255],
...
         [194, 194, 194, ..., 194, 194, 194],
         [194, 194, 194, ..., 194, 194, 194]]],


       [[[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [208, 208, 208, ..., 198, 196, 196],
         [208, 208, 208, ..., 198, 198, 196],
         [208, 208, 208, ..., 194, 198, 198]]],


       [[[194, 194, 194, ..., 200, 200, 200],
         [194, 194, 194, ..., 200, 200, 200],
         [194, 194, 194, ..., 200, 200, 200],
         ...,
         [200, 200, 200, ..., 208, 208, 208],
         [200, 200, 200, ..., 208, 208, 208],
         [200, 200, 200, ..., 208, 208, 208]]]], dtype=uint8)
Coordinates:
  * band           (band) <U5 20B 'Fmask'
  * x              (x) float64 13kB 3.995e+05 3.996e+05 ... 4.469e+05 4.469e+05
  * y              (y) float64 19kB 7.679e+06 7.679e+06 ... 7.606e+06 7.606e+06
    constellation  (time) <U3 3kB 'L30' 'L30' 'S30' 'S30' ... 'S30' 'L30' 'L30'
  * time           (time) datetime64[ns] 2kB 2024-03-01 ... 2024-10-11
Attributes:
    spec:        RasterSpec(epsg=32606, bounds=(399510, 7606260, 446940, 7679...
    crs:         epsg:32606
    transform:   | 30.00, 0.00, 399510.00|\n| 0.00,-30.00, 7679340.00|\n| 0.0...
    resolution:  30

In [7]:
# %%time
# import rasterio as rio
# with rio.Env(GDAL_HTTP_UNSAFESSL = 'YES') as env: # Works! 
#     fmask = fmask.load() 
# fmask

In [8]:
%%time
fmask = pputil.convertFmask(fmask, format = 'categories') # Without load before, got SSL timeout error
fmask

CPU times: total: 5.28 s
Wall time: 4.98 s


<xarray.DataArray 'stackstac-c5926b00bdc2dd1977b0699da7910e9d' (time: 245,
                                                                band: 1,
                                                                y: 2436, x: 1581)> Size: 944MB
array([[[[ 6,  6,  6, ..., 10, 10, 10],
         [ 6,  6,  6, ..., 10, 10, 10],
         [ 6,  6,  6, ..., 10, 10, 10],
         ...,
         [ 6,  6,  6, ...,  6,  6,  6],
         [ 6,  6,  6, ...,  6,  6,  6],
         [ 6,  6,  6, ...,  6,  6,  7]]],


       [[[10, 10, 10, ..., 10, 10, 10],
         [10, 10, 10, ..., 10, 10, 10],
         [10, 10, 10, ..., 10, 10, 10],
         ...,
         [10, 10, 10, ...,  6,  6,  6],
         [10, 10, 10, ...,  6,  6,  6],
         [10, 10, 10, ...,  6,  6,  6]]],


       [[[ 9,  9,  9, ..., 10, 10, 10],
         [ 9,  9,  9, ..., 10, 10, 10],
...
         [ 9,  9,  9, ...,  9,  9,  9],
         [ 9,  9,  9, ...,  9,  9,  9]]],


       [[[10, 10, 10, ..., 10, 10, 10],
         [10, 10, 10, ..., 10, 10, 10],
         [10, 10, 10, ..., 10, 10, 10],
         ...,
         [ 6,  6,  6, ...,  9,  7,  7],
         [ 6,  6,  6, ...,  9,  9,  7],
         [ 6,  6,  6, ...,  9,  9,  9]]],


       [[[ 9,  9,  9, ...,  8,  8,  8],
         [ 9,  9,  9, ...,  8,  8,  8],
         [ 9,  9,  9, ...,  8,  8,  8],
         ...,
         [ 8,  8,  8, ...,  6,  6,  6],
         [ 8,  8,  8, ...,  6,  6,  6],
         [ 8,  8,  8, ...,  6,  6,  6]]]], dtype=uint8)
Coordinates:
  * band           (band) <U5 20B 'Fmask'
  * x              (x) float64 13kB 3.995e+05 3.996e+05 ... 4.469e+05 4.469e+05
  * y              (y) float64 19kB 7.679e+06 7.679e+06 ... 7.606e+06 7.606e+06
    constellation  (time) <U3 3kB 'L30' 'L30' 'S30' 'S30' ... 'S30' 'L30' 'L30'
  * time           (time) datetime64[ns] 2kB 2024-03-01 ... 2024-10-11
Attributes:
    spec:        RasterSpec(epsg=32606, bounds=(399510, 7606260, 446940, 7679...
    crs:         epsg:32606
    transform:   | 30.00, 0.00, 399510.00|\n| 0.00,-30.00, 7679340.00|\n| 0.0...
    resolution:  30

In [18]:
%%time
pputil.downloadNC(fmask, r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\Fmask_V1\AA\Alaska Snow Depth\1 HLS Fmask from STAC\fmask_nc', 'fmask_AlaskaSite_' + str(year) + '.nc', form = 'hls')

CPU times: total: 109 ms
Wall time: 2.59 s


# Testing

In [10]:
# bboxLL = pputil.poly2bbox(area, 'lat/lon') # For STAC search

In [11]:
# bbox, epsg = pputil.poly2bbox(area, proj) # For stackstac

In [12]:
# gdalEnv, catalog = sutil.accessSTAC('https://cmr.earthdata.nasa.gov/stac/LPCLOUD')

In [13]:
# itemsL30 = catalog.search(bbox = bboxLL, datetime = f'{'2022-11-01'}/{'2022-12-31'}', collections = ['HLSL30.v2.0'], limit = 100).item_collection() 
# nL30 = len(itemsL30)
# print(nL30)

In [14]:
#itemsL30

In [15]:
# import numpy as np
# np.can_cast(type(np.uint8(255)), 'uint8')

In [16]:
# import stackstac as ss
# import os
# import pystac_client as pc

# # Temporal bounds
# year = 2018
# start = str(year) + '-01-01'
# end =  str(year) + '-12-31'

# # Spatial bounds
# bboxLL = (np.float64(-149.53623682903373), np.float64(68.55062861124203), np.float64(-148.30140593046852), np.float64(69.2185427988887))
# epsg = 32606
# bbox = (np.float64(399535.48481308576), np.float64(7606266.209112197), np.float64(446924.35747663863), np.float64(7679311.477803782))

# # STAC settings
# gdalEnv = ss.DEFAULT_GDAL_ENV.updated(dict(
#     GDAL_DISABLE_READDIR_ON_OPEN = 'TRUE',
#     GDAL_HTTP_COOKIEFILE = os.path.expanduser('~/cookies.txt'),
#     GDAL_HTTP_COOKIEJAR = os.path.expanduser('~/cookies.txt'),
#     GDAL_HTTP_MAX_RETRY = 10,
#     GDAL_HTTP_RETRY_DELAY = 15, 
#     GDAL_HTTP_UNSAFESSL = 'YES')) # Will pass to Dask cluster workers

# catalog = pc.Client.open('https://cmr.earthdata.nasa.gov/stac/LPCLOUD')

# # Create stack
# items = catalog.search(bbox = bboxLL, datetime = f'{start}/{end}', collections = ['HLSL30.v2.0'], limit = 100).item_collection() 

# stack = ss.stack(items, assets = ['Fmask'], epsg = epsg, resolution = 30, bounds = bbox, dtype = 'uint8', fill_value = np.uint8(255), #np.uint8(255)
#     rescale = False, gdal_env = gdalEnv)
# stack

In [17]:
# %%time
# stack = stack.load()
# stack